In [1]:
from google.colab import drive
from os import listdir
from os.path import isfile, join
from scipy.io import loadmat
from random import randint
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, classification_report

import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K
import pandas as pd
import numpy as np
import random

import datetime
import os

In [2]:
# Establecemos conexión con Google Drive para acceder a la carpeta de datasets
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Guardamos la dirección de los datasets (experimentos) en una variable para utilizala posteriormente
dataset_datos_dir = 'drive/MyDrive/DATOS_EXPERIMENTALES_JACKET/DATOS'

In [4]:
# Obtenemos una lista con todos los nombres de los archivos relacionados a los experimentos realizados
dataset_datos_files = [dataset_datos_dir+'/'+filename for filename in listdir(dataset_datos_dir) if isfile(join(dataset_datos_dir, filename))]

In [5]:
# Creamos listas en donde se almacenarán los archivos relacionados a experimentos de fisura y experimentos de perno flojo respectivamente
datos_fisura = []
datos_pernoflojo = []
datos_replica = []
datos_normal = []

In [6]:
# Iteramos los nombres de todos los archivos en la lista general previamente creada
for filename in listdir(dataset_datos_dir):
  # Obtenemos el nombre del archivo tal cual, sin direcciones previas
  if isfile(join(dataset_datos_dir, filename)):
    nombre_archivo = filename.split('.')[0]
    # El estado de fisura se asocia al número 3 en los nombres de los archivos, si cumple la condición el archivo, guardamos en la lista datos_fisura
    if (nombre_archivo.split('_')[0] == '3'):
      datos_fisura.append(dataset_datos_dir + '/' + filename)
    # El estado de fisura se asocia al número 4 en los nombres de los archivos, si cumple la condición el archivo, guardamos en la lista datos_pernoflojo
    elif (nombre_archivo.split('_')[0] == '4'):
      datos_pernoflojo.append(dataset_datos_dir + '/' + filename)
    # El estado de fisura se asocia al número 4 en los nombres de los archivos, si cumple la condición el archivo, guardamos en la lista datos_pernoflojo
    elif (nombre_archivo.split('_')[0] == '1'):
      datos_normal.append(dataset_datos_dir + '/' + filename)
    # El estado de fisura se asocia al número 4 en los nombres de los archivos, si cumple la condición el archivo, guardamos en la lista datos_pernoflojo
    elif (nombre_archivo.split('_')[0] == '2'):
      datos_replica.append(dataset_datos_dir + '/' + filename)

Datos Fisura

In [7]:
# Se crean 2 listas, una asociada al número de experimento y otra asociada al nivel de white noise presente durante el experimento 
numero_experimentos_fisura = []
amplitud_experimentos_fisura = []

In [8]:
# Iteramos los nombres de los archivos dentro del conjunto que contiene los datos de fisura
for nombre_archivo in datos_fisura:
  # Obtenemos el nombre del experimento
  nombre_experimento = nombre_archivo.split('/')[4]
  # Obtenemos el número del experimento
  numero_experimento = nombre_experimento.split('_')[1]
  # Obtenemos el nivel de white noise del experimento
  amplitud_experimento = nombre_experimento.split('.')[0].split('_')[2].replace('A', '')
  # Agregamos el número del experimento a la lista numero_experimentos_fisura creada anteriormente
  numero_experimentos_fisura.append(int(numero_experimento))
  # Agregamos el nivel de white noise del experimento a la lista amplitud_experimentos_fisura creada anteriormente
  if amplitud_experimento == '05':
    amplitud_experimento = 0.5
    amplitud_experimentos_fisura.append(amplitud_experimento)
  else:
    amplitud_experimentos_fisura.append(int(amplitud_experimento))

In [9]:
# Creamos un DataFrame vacío
df_fisura = pd.DataFrame()

In [10]:
# Iteramos por rango la lista datos_fisura
for indice in range(len(datos_fisura)):
  # Obtenemos la dirección dentro de la iteración correspondiente
  direccion = datos_fisura[indice]
  # Transformamos el .mat a un formato en el que se pueda transformar a DataFrame
  mat = loadmat(direccion)
  df = pd.DataFrame(mat['data'])
  # Para todos los valores del experimento, creamos columnas en donde sus valores se asocian con el nivel de wn y el # de experimento
  df['#_exp'] = numero_experimentos_fisura[indice]
  df['amplitud'] = amplitud_experimentos_fisura[indice]
  # Colocamos al final del DataFrame df_fisura creado el nuevo DataFrame 
  df_fisura = pd.concat([df_fisura, df], axis = 0)

In [11]:
# Si dividimos para 99097 el df_fisura deberíamos obtener 20 experimentos en total
len(df_fisura)/99097

20.0

In [12]:
# Establecemos 2 listas, una con los posibles experimentos y otra con los posibles niveles de white noise, lo cual nos ayudará a segmentar los datos
experimentos = [1, 2, 3, 4, 5]
wns = [0.5, 1, 2, 3]
# Creamos una lista en donde pondremos todas las imágenes creadas, segmentadas por experimento y white noise
arreglo_imagenes_por_experimento_fisura = []

# Iteramos por experimento y por nivel de white noise
for i in experimentos:
  for j in wns:
    # Creamos filtros que nos permiten identificar el experimento por su número asociado y por su nivel de white noise dentro del conjunto de datos
    filter1 = df_fisura["#_exp"] == i
    filter2 = df_fisura["amplitud"] == j
    # Filtramos y deshacemos las filas que no corresponden a la búsqueda
    dataset_experimento = df_fisura.where(filter1 & filter2).dropna()
    # Creamos arreglo en donde almacenaremos listas de imágenes asociadas a cada columna en un experimento individual
    arreglo_matrices_asociadas = []

    # En el conjunto filtrado, iteramos por cada columna (de la columna 0 a la 23)
    for n in range(24):
      # Establecemos un límite inferior y un límite superior
      indice_inicio = 0
      indice_fin = 256
      # Del conjunto filtrado, obtenemos un subconjunto con muestras cada 6 pasos
      columna_cada_6 = dataset_experimento[n][::6]
      # Creamos una lista en donde guardaremos todas las imágenes creadas dentro de la columna en la que se esta iterando
      matrices_columna = []
    
      # Recorremos en valores de 256 la columna del subconjunto creado para generar las imágenes 16x16 correspondientes 
      while indice_fin < len(columna_cada_6):
        # Guardamos el conjunto de 256 datos dentro de un arreglo
        vector = columna_cada_6[indice_inicio:indice_fin]
        # Redimensionamos el arreglo (de 1x256 a 16x16)
        matriz = vector.to_numpy().reshape((16, 16))
        # Agregamos la imagen dentro de la lista matrices_columna
        matrices_columna.append(matriz)
        # Establecemos el nuevo límite inferior y superior en donde se encontrarán los datos de mi nueva imagen 16x16
        indice_inicio = indice_fin
        indice_fin = indice_fin + 256
      
      # Agregamos las listas de imágenes 16x16 asociadas a una columna a arreglo_matrices_asociadas
      arreglo_matrices_asociadas.append(matrices_columna)
    
    # Agregamos arreglo_matrices_asociadas a arreglos_imagenes_por_experimento, en donde tenemos listas de listas asociadas a un experimento individual
    arreglo_imagenes_por_experimento_fisura.append(arreglo_matrices_asociadas)

In [13]:
# Creamos un arreglos general para todas las imágenes de dimensionamiento (24, 16, 16)
arreglo_imagenes_fisura = []

# Iteramos cada experimento en la lista de experimentos creada previamente
for experimento in arreglo_imagenes_por_experimento_fisura:
  # Iteramos en un rango de 64 para usarlo posteriormente
  for idx in range(64):
    # Para cada iteración se crea el arreglo en donde se agregaran las capas correctas de la imagen de dimensionamniento (24, 16, 16)
    imagen = []
    # Iteramos para cada fila en el experimento (son 24 filas)
    for fila in experimento:
      # Asociamos por índice los valores de cada lista de imágenes para establecer la relación correcta y agregamos al arreglo creado
      capa_imagen = fila[idx]
      imagen.append(capa_imagen)

    # Transformamos la lista de 24 capas a un arreglo numérico
    arreglo_imagenes_fisura.append(np.array(imagen))

In [14]:
# Revisamos el número de imágenes creadas
len(arreglo_imagenes_fisura)

1280

Datos Perno Flojo

In [15]:
# Se crean 2 listas, una asociada al número de experimento y otra asociada al nivel de white noise presente durante el experimento 
numero_experimentos_pernoflojo = []
amplitud_experimentos_pernoflojo = []

In [16]:
# Iteramos los nombres de los archivos dentro del conjunto que contiene los datos de fisura
for nombre_archivo in datos_pernoflojo:
  # Obtenemos el nombre del experimento
  nombre_experimento = nombre_archivo.split('/')[4]
  # Obtenemos el número del experimento
  numero_experimento = nombre_experimento.split('_')[1]
  # Obtenemos el nivel de white noise del experimento
  amplitud_experimento = nombre_experimento.split('.')[0].split('_')[2].replace('A', '')
  # Agregamos el número del experimento a la lista numero_experimentos_pernoflojo creada anteriormente
  numero_experimentos_pernoflojo.append(int(numero_experimento))
  # Agregamos el nivel de white noise del experimento a la lista amplitud_experimentos_pernoflojo creada anteriormente
  if amplitud_experimento == '05':
    amplitud_experimento = 0.5
    amplitud_experimentos_pernoflojo.append(amplitud_experimento)
  else:
    amplitud_experimentos_pernoflojo.append(int(amplitud_experimento))

In [17]:
# Creamos un DataFrame vacío
df_pernoflojo = pd.DataFrame()

In [18]:
# Iteramos por rango la lista datos_fisura
for indice in range(len(datos_pernoflojo)):
  # Obtenemos la dirección dentro de la iteración correspondiente
  direccion = datos_pernoflojo[indice]
  # Transformamos el .mat a un formato en el que se pueda transformar a DataFrame
  mat = loadmat(direccion)
  df = pd.DataFrame(mat['data'])
  # Para todos los valores del experimento, creamos columnas en donde sus valores se asocian con el nivel de wn y el # de experimento
  df['#_exp'] = numero_experimentos_pernoflojo[indice]
  df['amplitud'] = amplitud_experimentos_pernoflojo[indice]
  # Colocamos al final del DataFrame df_fisura creado el nuevo DataFrame 
  df_pernoflojo = pd.concat([df_pernoflojo, df], axis = 0)

In [19]:
# Si dividimos para 99097 el df_pernoflojo deberíamos obtener 20 experimentos en total
len(df_pernoflojo)/99097

20.0

In [20]:
# Establecemos 2 listas, una con los posibles experimentos y otra con los posibles niveles de white noise, lo cual nos ayudará a segmentar los datos
experimentos = [1, 2, 3, 4, 5]
wns = [0.5, 1, 2, 3]
# Creamos una lista en donde pondremos todas las imágenes creadas, segmentadas por experimento y white noise
arreglo_imagenes_por_experimento_pernoflojo = []

# Iteramos por experimento y por nivel de white noise
for i in experimentos:
  for j in wns:
    # Creamos filtros que nos permiten identificar el experimento por su número asociado y por su nivel de white noise dentro del conjunto de datos
    filter1 = df_pernoflojo["#_exp"] == i
    filter2 = df_pernoflojo["amplitud"] == j
    # Filtramos y deshacemos las filas que no corresponden a la búsqueda
    dataset_experimento = df_pernoflojo.where(filter1 & filter2).dropna()
    # Creamos arreglo en donde almacenaremos listas de imágenes asociadas a cada columna en un experimento individual
    arreglo_matrices_asociadas = []

    # En el conjunto filtrado, iteramos por cada columna (de la columna 0 a la 23)
    for n in range(24):
      # Establecemos un límite inferior y un límite superior
      indice_inicio = 0
      indice_fin = 256
      # Del conjunto filtrado, obtenemos un subconjunto con muestras cada 6 pasos
      columna_cada_6 = dataset_experimento[n][::6]
      # Creamos una lista en donde guardaremos todas las imágenes creadas dentro de la columna en la que se esta iterando
      matrices_columna = []
    
      # Recorremos en valores de 256 la columna del subconjunto creado para generar las imágenes 16x16 correspondientes 
      while indice_fin < len(columna_cada_6):
        # Guardamos el conjunto de 256 datos dentro de un arreglo
        vector = columna_cada_6[indice_inicio:indice_fin]
        # Redimensionamos el arreglo (de 1x256 a 16x16)
        matriz = vector.to_numpy().reshape((16, 16))
        # Agregamos la imagen dentro de la lista matrices_columna
        matrices_columna.append(matriz)
        # Establecemos el nuevo límite inferior y superior en donde se encontrarán los datos de mi nueva imagen 16x16
        indice_inicio = indice_fin
        indice_fin = indice_fin + 256
      
      # Agregamos las listas de imágenes 16x16 asociadas a una columna a arreglo_matrices_asociadas
      arreglo_matrices_asociadas.append(matrices_columna)
    
    # Agregamos arreglo_matrices_asociadas a arreglos_imagenes_por_experimento, en donde tenemos listas de listas asociadas a un experimento individual
    arreglo_imagenes_por_experimento_pernoflojo.append(arreglo_matrices_asociadas)

In [21]:
# Creamos un arreglos general para todas las imágenes de dimensionamiento (24, 16, 16)
arreglo_imagenes_pernoflojo = []

# Iteramos cada experimento en la lista de experimentos creada previamente
for experimento_i in arreglo_imagenes_por_experimento_pernoflojo:
  # Iteramos en un rango de 64 para usarlo posteriormente
  for idx in range(64):
    # Para cada iteración se crea el arreglo en donde se agregaran las capas correctas de la imagen de dimensionamniento (24, 16, 16)
    imagen = []
    # Iteramos para cada fila en el experimento (son 24 filas)
    for fila in experimento_i:
      # Asociamos por índice los valores de cada lista de imágenes para establecer la relación correcta y agregamos al arreglo creado
      capa_imagen = fila[idx]
      imagen.append(capa_imagen)

    # Transformamos la lista de 24 capas a un arreglo numérico
    arreglo_imagenes_pernoflojo.append(np.array(imagen))

In [22]:
# Revisamos el número de imágenes creadas
len(arreglo_imagenes_pernoflojo)

1280

Datos Réplica

In [23]:
# Se crean 2 listas, una asociada al número de experimento y otra asociada al nivel de white noise presente durante el experimento 
numero_experimentos_replica = []
amplitud_experimentos_replica = []

In [24]:
# Iteramos los nombres de los archivos dentro del conjunto que contiene los datos de fisura
for nombre_archivo in datos_replica:
  # Obtenemos el nombre del experimento
  nombre_experimento = nombre_archivo.split('/')[4]
  # Obtenemos el número del experimento
  numero_experimento = nombre_experimento.split('_')[1]
  # Obtenemos el nivel de white noise del experimento
  amplitud_experimento = nombre_experimento.split('.')[0].split('_')[2].replace('A', '')
  # Agregamos el número del experimento a la lista numero_experimentos_pernoflojo creada anteriormente
  numero_experimentos_replica.append(int(numero_experimento))
  # Agregamos el nivel de white noise del experimento a la lista amplitud_experimentos_pernoflojo creada anteriormente
  if amplitud_experimento == '05':
    amplitud_experimento = 0.5
    amplitud_experimentos_replica.append(amplitud_experimento)
  else:
    amplitud_experimentos_replica.append(int(amplitud_experimento))

In [25]:
# Creamos un DataFrame vacío
df_replica = pd.DataFrame()

In [26]:
# Iteramos por rango la lista datos_fisura
for indice in range(len(datos_replica)):
  # Obtenemos la dirección dentro de la iteración correspondiente
  direccion = datos_replica[indice]
  # Transformamos el .mat a un formato en el que se pueda transformar a DataFrame
  mat = loadmat(direccion)
  df = pd.DataFrame(mat['data'])
  # Para todos los valores del experimento, creamos columnas en donde sus valores se asocian con el nivel de wn y el # de experimento
  df['#_exp'] = numero_experimentos_replica[indice]
  df['amplitud'] = amplitud_experimentos_replica[indice]
  # Colocamos al final del DataFrame df_fisura creado el nuevo DataFrame 
  df_replica = pd.concat([df_replica, df], axis = 0)

In [27]:
# Si dividimos para 99097 el df_pernoflojo deberíamos obtener 20 experimentos en total
len(df_pernoflojo)/99097

20.0

In [28]:
# Establecemos 2 listas, una con los posibles experimentos y otra con los posibles niveles de white noise, lo cual nos ayudará a segmentar los datos
experimentos = [1, 2, 3, 4, 5]
wns = [0.5, 1, 2, 3]
# Creamos una lista en donde pondremos todas las imágenes creadas, segmentadas por experimento y white noise
arreglo_imagenes_por_experimento_replica = []

# Iteramos por experimento y por nivel de white noise
for i in experimentos:
  for j in wns:
    # Creamos filtros que nos permiten identificar el experimento por su número asociado y por su nivel de white noise dentro del conjunto de datos
    filter1 = df_replica["#_exp"] == i
    filter2 = df_replica["amplitud"] == j
    # Filtramos y deshacemos las filas que no corresponden a la búsqueda
    dataset_experimento = df_replica.where(filter1 & filter2).dropna()
    # Creamos arreglo en donde almacenaremos listas de imágenes asociadas a cada columna en un experimento individual
    arreglo_matrices_asociadas = []

    # En el conjunto filtrado, iteramos por cada columna (de la columna 0 a la 23)
    for n in range(24):
      # Establecemos un límite inferior y un límite superior
      indice_inicio = 0
      indice_fin = 256
      # Del conjunto filtrado, obtenemos un subconjunto con muestras cada 6 pasos
      columna_cada_6 = dataset_experimento[n][::6]
      # Creamos una lista en donde guardaremos todas las imágenes creadas dentro de la columna en la que se esta iterando
      matrices_columna = []
    
      # Recorremos en valores de 256 la columna del subconjunto creado para generar las imágenes 16x16 correspondientes 
      while indice_fin < len(columna_cada_6):
        # Guardamos el conjunto de 256 datos dentro de un arreglo
        vector = columna_cada_6[indice_inicio:indice_fin]
        # Redimensionamos el arreglo (de 1x256 a 16x16)
        matriz = vector.to_numpy().reshape((16, 16))
        # Agregamos la imagen dentro de la lista matrices_columna
        matrices_columna.append(matriz)
        # Establecemos el nuevo límite inferior y superior en donde se encontrarán los datos de mi nueva imagen 16x16
        indice_inicio = indice_fin
        indice_fin = indice_fin + 256
      
      # Agregamos las listas de imágenes 16x16 asociadas a una columna a arreglo_matrices_asociadas
      arreglo_matrices_asociadas.append(matrices_columna)
    
    # Agregamos arreglo_matrices_asociadas a arreglos_imagenes_por_experimento, en donde tenemos listas de listas asociadas a un experimento individual
    arreglo_imagenes_por_experimento_replica.append(arreglo_matrices_asociadas)

In [29]:
# Creamos un arreglos general para todas las imágenes de dimensionamiento (24, 16, 16)
arreglo_imagenes_replica = []

# Iteramos cada experimento en la lista de experimentos creada previamente
for experimento_i in arreglo_imagenes_por_experimento_replica:
  # Iteramos en un rango de 64 para usarlo posteriormente
  for idx in range(64):
    # Para cada iteración se crea el arreglo en donde se agregaran las capas correctas de la imagen de dimensionamniento (24, 16, 16)
    imagen = []
    # Iteramos para cada fila en el experimento (son 24 filas)
    for fila in experimento_i:
      # Asociamos por índice los valores de cada lista de imágenes para establecer la relación correcta y agregamos al arreglo creado
      capa_imagen = fila[idx]
      imagen.append(capa_imagen)

    # Transformamos la lista de 24 capas a un arreglo numérico
    arreglo_imagenes_replica.append(np.array(imagen))

In [30]:
# Revisamos el número de imágenes creadas
len(arreglo_imagenes_replica)

1280

Datos Normal

In [31]:
# Se crean 2 listas, una asociada al número de experimento y otra asociada al nivel de white noise presente durante el experimento 
numero_experimentos_normal = []
amplitud_experimentos_normal = []

In [32]:
# Iteramos los nombres de los archivos dentro del conjunto que contiene los datos de fisura
for nombre_archivo in datos_normal:
  # Obtenemos el nombre del experimento
  nombre_experimento = nombre_archivo.split('/')[4]
  # Obtenemos el número del experimento
  numero_experimento = nombre_experimento.split('_')[1]
  # Obtenemos el nivel de white noise del experimento
  amplitud_experimento = nombre_experimento.split('.')[0].split('_')[2].replace('A', '')
  # Agregamos el número del experimento a la lista numero_experimentos_pernoflojo creada anteriormente
  numero_experimentos_normal.append(int(numero_experimento))
  # Agregamos el nivel de white noise del experimento a la lista amplitud_experimentos_pernoflojo creada anteriormente
  if amplitud_experimento == '05':
    amplitud_experimento = 0.5
    amplitud_experimentos_normal.append(amplitud_experimento)
  else:
    amplitud_experimentos_normal.append(int(amplitud_experimento))

In [33]:
# Creamos un DataFrame vacío
df_normal = pd.DataFrame()

In [34]:
# Iteramos por rango la lista datos_fisura
for indice in range(len(datos_normal)):
  # Obtenemos la dirección dentro de la iteración correspondiente
  direccion = datos_normal[indice]
  # Transformamos el .mat a un formato en el que se pueda transformar a DataFrame
  mat = loadmat(direccion)
  df = pd.DataFrame(mat['data'])
  # Para todos los valores del experimento, creamos columnas en donde sus valores se asocian con el nivel de wn y el # de experimento
  df['#_exp'] = numero_experimentos_normal[indice]
  df['amplitud'] = amplitud_experimentos_normal[indice]
  # Colocamos al final del DataFrame df_fisura creado el nuevo DataFrame 
  df_normal = pd.concat([df_normal, df], axis = 0)

In [35]:
# Si dividimos para 99097 el df_pernoflojo deberíamos obtener 20 experimentos en total NO SABEMOS TODAVIA
len(df_normal)/99097

40.0

In [36]:
# Establecemos 2 listas, una con los posibles experimentos y otra con los posibles niveles de white noise, lo cual nos ayudará a segmentar los datos
experimentosn = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
wns = [0.5, 1, 2, 3]
# Creamos una lista en donde pondremos todas las imágenes creadas, segmentadas por experimento y white noise
arreglo_imagenes_por_experimento_normal = []

# Iteramos por experimento y por nivel de white noise
for i in experimentosn:
  for j in wns:
    # Creamos filtros que nos permiten identificar el experimento por su número asociado y por su nivel de white noise dentro del conjunto de datos
    filter1 = df_normal["#_exp"] == i
    filter2 = df_normal["amplitud"] == j
    # Filtramos y deshacemos las filas que no corresponden a la búsqueda
    dataset_experimento = df_normal.where(filter1 & filter2).dropna()
    # Creamos arreglo en donde almacenaremos listas de imágenes asociadas a cada columna en un experimento individual
    arreglo_matrices_asociadas = []

    # En el conjunto filtrado, iteramos por cada columna (de la columna 0 a la 23)
    for n in range(24):
      # Establecemos un límite inferior y un límite superior
      indice_inicio = 0
      indice_fin = 256
      # Del conjunto filtrado, obtenemos un subconjunto con muestras cada 6 pasos
      columna_cada_6 = dataset_experimento[n][::6]
      # Creamos una lista en donde guardaremos todas las imágenes creadas dentro de la columna en la que se esta iterando
      matrices_columna = []
    
      # Recorremos en valores de 256 la columna del subconjunto creado para generar las imágenes 16x16 correspondientes 
      while indice_fin < len(columna_cada_6):
        # Guardamos el conjunto de 256 datos dentro de un arreglo
        vector = columna_cada_6[indice_inicio:indice_fin]
        # Redimensionamos el arreglo (de 1x256 a 16x16)
        matriz = vector.to_numpy().reshape((16, 16))
        # Agregamos la imagen dentro de la lista matrices_columna
        matrices_columna.append(matriz)
        # Establecemos el nuevo límite inferior y superior en donde se encontrarán los datos de mi nueva imagen 16x16
        indice_inicio = indice_fin
        indice_fin = indice_fin + 256
      
      # Agregamos las listas de imágenes 16x16 asociadas a una columna a arreglo_matrices_asociadas
      arreglo_matrices_asociadas.append(matrices_columna)
    
    # Agregamos arreglo_matrices_asociadas a arreglos_imagenes_por_experimento, en donde tenemos listas de listas asociadas a un experimento individual
    arreglo_imagenes_por_experimento_normal.append(arreglo_matrices_asociadas)

In [37]:
# Creamos un arreglos general para todas las imágenes de dimensionamiento (24, 16, 16)
arreglo_imagenes_normal = []

# Iteramos cada experimento en la lista de experimentos creada previamente
for experimento_i in arreglo_imagenes_por_experimento_normal:
  # Iteramos en un rango de 64 para usarlo posteriormente
  for idx in range(64):
    # Para cada iteración se crea el arreglo en donde se agregaran las capas correctas de la imagen de dimensionamniento (24, 16, 16)
    imagen = []
    # Iteramos para cada fila en el experimento (son 24 filas)
    for fila in experimento_i:
      # Asociamos por índice los valores de cada lista de imágenes para establecer la relación correcta y agregamos al arreglo creado
      capa_imagen = fila[idx]
      imagen.append(capa_imagen)

    # Transformamos la lista de 24 capas a un arreglo numérico
    arreglo_imagenes_normal.append(np.array(imagen))

In [38]:
# Revisamos el número de imágenes creadas
len(arreglo_imagenes_normal)

2560

Pruebas

In [39]:
def load_model(model_path):
  """
  Carga un modelo guardado previamente.
  """
  print(f'Cargando modelo guardado de: {model_path}...')
  model = tf.keras.models.load_model(model_path)

  return model

In [40]:
# Almacenamos el modelo en una variable
modelo_7_loaded = load_model('drive/MyDrive/modelos/20210712-17041626109450-modelo_SNN_TurbinaJacket_Sano_Fallo.h5')

Cargando modelo guardado de: drive/MyDrive/modelos/20210712-17041626109450-modelo_SNN_TurbinaJacket_Sano_Fallo.h5...


In [46]:
def prueba_prediccion(modelo, arreglo_img1_fisura, arreglo_img2_pernoflojo, arreglo_img3_replica, arreglo_img4_normal):

  # Establecemos el modo de manera aleatoria
  modo = random.randint(1, 10)

  # Inicializamos las imágenes de prueba
  imagen_prueba_1 = []
  imagen_prueba_2 = []

  # Establecemos las condiciones de modo para generar el par de imágenes a comparar
  if modo == 1:
    imagen_prueba_1 = random.choice(arreglo_img1_fisura)
    imagen_prueba_2 = random.choice(arreglo_img1_fisura)
  elif modo == 2:
    imagen_prueba_1 = random.choice(arreglo_img2_pernoflojo)
    imagen_prueba_2 = random.choice(arreglo_img2_pernoflojo)
  elif modo == 3:
    imagen_prueba_1 = random.choice(arreglo_img1_fisura)
    imagen_prueba_2 = random.choice(arreglo_img2_pernoflojo)
  elif modo == 4:
    imagen_prueba_1 = random.choice(arreglo_img1_fisura)
    imagen_prueba_2 = random.choice(arreglo_img3_replica)
  elif modo == 5:
    imagen_prueba_1 = random.choice(arreglo_img1_fisura)
    imagen_prueba_2 = random.choice(arreglo_img4_normal)
  elif modo == 6:
    imagen_prueba_1 = random.choice(arreglo_img2_pernoflojo)
    imagen_prueba_2 = random.choice(arreglo_img3_replica)
  elif modo == 7:
    imagen_prueba_1 = random.choice(arreglo_img2_pernoflojo)
    imagen_prueba_2 = random.choice(arreglo_img4_normal)
  elif modo == 8:
    imagen_prueba_1 = random.choice(arreglo_img3_replica)
    imagen_prueba_2 = random.choice(arreglo_img3_replica)
  elif modo == 9:
    imagen_prueba_1 = random.choice(arreglo_img3_replica)
    imagen_prueba_2 = random.choice(arreglo_img4_normal)
  elif modo == 10:
    imagen_prueba_1 = random.choice(arreglo_img4_normal)
    imagen_prueba_2 = random.choice(arreglo_img4_normal)

  # Expandimos dimensiones para poder ingresar las imágenes al modelo (relacionado a número de batches)
  imagen_prueba_1 = np.expand_dims(imagen_prueba_1, axis=-1)
  imagen_prueba_2 = np.expand_dims(imagen_prueba_2, axis=-1)

  imagen_prueba_1 = np.expand_dims(imagen_prueba_1, axis=0)
  imagen_prueba_2 = np.expand_dims(imagen_prueba_2, axis=0)

  # Realizamos predicción
  prediccion_prueba = modelo.predict([imagen_prueba_1, imagen_prueba_2])[0][0]
  prediccion_prueba_redondeada = round(modelo.predict([imagen_prueba_1, imagen_prueba_2])[0][0])

  # Establecemos condiciones para mostrar distintos mensajes de acuerdo a la combinación escogida aleatoriamente
  if modo == 1:
    print("---------- Resultados ----------")
    print("Primera imagen: fisura")
    print("Segunda imagen: fisura")
    print("Predicción: similares") if prediccion_prueba_redondeada == 1  else print("Predicción: no similares")
    print("Descripción real: similares")
    print(f"Porcentaje: '{prediccion_prueba*100:.2f}%")
  if modo == 2:
    print("---------- Resultados ----------")
    print("Primera imagen: perno flojo")
    print("Segunda imagen: perno flojo")
    print("Predicción: similares") if prediccion_prueba_redondeada == 1 else print("Predicción: no similares")
    print("Descripción real: similares")
    print(f"Porcentaje: '{prediccion_prueba*100:.2f}%")
  if modo == 3:
    print("---------- Resultados ----------")
    print("Primera imagen: fisura")
    print("Segunda imagen: perno flojo")
    print("Predicción: similares") if prediccion_prueba_redondeada == 1 else print("Predicción: no similares")
    print("Descripción real: similares")
    print(f"Porcentaje: '{prediccion_prueba*100:.2f}%")
  if modo == 4:
    print("---------- Resultados ----------")
    print("Primera imagen: fisura")
    print("Segunda imagen: réplica")
    print("Predicción: similares") if prediccion_prueba_redondeada == 1 else print("Predicción: no similares")
    print("Descripción real: no similares")
    print(f"Porcentaje: '{prediccion_prueba*100:.2f}%")
  if modo == 5:
    print("---------- Resultados ----------")
    print("Primera imagen: fisura")
    print("Segunda imagen: normal")
    print("Predicción: similares") if prediccion_prueba_redondeada == 1 else print("Predicción: no similares")
    print("Descripción real: no similares")
    print(f"Porcentaje: '{prediccion_prueba*100:.2f}%")
  if modo == 6:
    print("---------- Resultados ----------")
    print("Primera imagen: perno flojo")
    print("Segunda imagen: réplica")
    print("Predicción: similares") if prediccion_prueba_redondeada == 1 else print("Predicción: no similares")
    print("Descripción real: no similares")
    print(f"Porcentaje: '{prediccion_prueba*100:.2f}%")
  if modo == 7:
    print("---------- Resultados ----------")
    print("Primera imagen: pernoflojo")
    print("Segunda imagen: normal")
    print("Predicción: similares") if prediccion_prueba_redondeada == 1 else print("Predicción: no similares")
    print("Descripción real: no similares")
    print(f"Porcentaje: '{prediccion_prueba*100:.2f}%")
  if modo == 8:
    print("---------- Resultados ----------")
    print("Primera imagen: réplica")
    print("Segunda imagen: réplica")
    print("Predicción: similares") if prediccion_prueba_redondeada == 1 else print("Predicción: no similares")
    print("Descripción real: similares")
    print(f"Porcentaje: '{prediccion_prueba*100:.2f}%")
  if modo == 9:
    print("---------- Resultados ----------")
    print("Primera imagen: réplica")
    print("Segunda imagen: normal")
    print("Predicción: similares") if prediccion_prueba_redondeada == 1 else print("Predicción: no similares")
    print("Descripción real: similares")
    print(f"Porcentaje: '{prediccion_prueba*100:.2f}%")
  if modo == 10:
    print("---------- Resultados ----------")
    print("Primera imagen: normal")
    print("Segunda imagen: normal")
    print("Predicción: similares") if prediccion_prueba_redondeada == 1 else print("Predicción: no similares")
    print("Descripción real: similares")
    print(f"Porcentaje: '{prediccion_prueba*100:.2f}%")

In [57]:
# Realizamos pruebas
prueba_prediccion(modelo_7_loaded, arreglo_imagenes_fisura, arreglo_imagenes_pernoflojo, arreglo_imagenes_replica, arreglo_imagenes_normal)

---------- Resultados ----------
Primera imagen: fisura
Segunda imagen: fisura
Predicción: similares
Descripción real: similares
Porcentaje: '78.42%
